<a href="https://colab.research.google.com/github/fginter/ainl_2020_tutorial/blob/main/parser_tnpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Turku Neural Parser Pipeline - Python module version

* This is a basic tutorial for running the parser pipeline under Google Colab
* Many new properties added for the AINL2020 tutorial
  * Parser changed to Udify (87->91% LAS improvement)
  * Code restructured so as to be importable as a module and can be installed using pip
  * No longer depends on Tensorflow
* Makes it possible for anyone to run the parser with GPU acceleration


# Install

* Pre-built wheel, not yet in PyPi
* Only Finnish and English models in this tutorial, more coming!

## Install the parser package (takes its time)

`pip3 install http://dl.turkunlp.org/turku-parser-models/turku_neural_parser-0.3-py3-none-any.whl`

## Download and unpack the model

`wget http://dl.turkunlp.org/turku-parser-models/models_fi_tdt_v2.7.tar.gz ; tar zxvf models_fi_tdt.tar.gz`

...and you are good to go!

# Prerequisites:

* The models here are tested with torch 1.7
* It might be that at some point this notebook will break
* If that happens, try to install torch 1.7


In [ ]:
!nvcc --version
!python -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Python 3.6.9


In [2]:
!wget -nc http://dl.turkunlp.org/turku-parser-models/turku_neural_parser-0.3-py3-none-any.whl
!pip3 install turku_neural_parser-0.3-py3-none-any.whl

File ‘turku_neural_parser-0.3-py3-none-any.whl’ already there; not retrieving.

Processing ./turku_neural_parser-0.3-py3-none-any.whl
     |████████████████████████████████| 7.0MB 6.4MB/s 
     |████████████████████████████████| 675kB 49.5MB/s 
     |████████████████████████████████| 204kB 47.5MB/s 
     |████████████████████████████████| 7.6MB 24.6MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 307kB 48.6MB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
     |████████████████████████████████| 3.8MB 50.3MB/s 
     |████████████████████████████████| 1.1MB 42.8MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 2.5MB 50.2MB/s 
     |████████████████████████████████| 133kB 46.8MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 245kB 56.4MB/s 
     |████████████████████████████

In [3]:
!wget -nc http://dl.turkunlp.org/turku-parser-models/models_fi_tdt_v2.7.tar.gz
!tar zxvf models_fi_tdt_v2.7.tar.gz

--2020-12-10 19:38:11--  http://dl.turkunlp.org/turku-parser-models/models_fi_tdt_v2.7.tar.gz
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 590211933 (563M) [application/octet-stream]
Saving to: ‘models_fi_tdt_v2.7.tar.gz’

models_fi_tdt_v2.7. 100%[===================>] 562.87M  11.0MB/s    in 49s     

2020-12-10 19:39:01 (11.5 MB/s) - ‘models_fi_tdt_v2.7.tar.gz’ saved [590211933/590211933]

models_fi_tdt_v2.7/
models_fi_tdt_v2.7/pipelines.yaml
models_fi_tdt_v2.7/Tokenizer/
models_fi_tdt_v2.7/Tokenizer/tokenizer.udpipe
models_fi_tdt_v2.7/Lemmatizer/
models_fi_tdt_v2.7/Lemmatizer/big_lemma_cache.tsv
models_fi_tdt_v2.7/Lemmatizer/lemma_cache.tsv
models_fi_tdt_v2.7/Lemmatizer/lemmatizer.pt
models_fi_tdt_v2.7/Udify/
models_fi_tdt_v2.7/Udify/model.tar.gz


# Running the parser

* Every model can specify many processing pipelines
* These are in `modeldir/pipelines.yaml`
* `parse_plaintext`is the default

In [5]:
from tnparser.pipeline import read_pipelines, Pipeline

available_pipelines=read_pipelines("models_fi_tdt_v2.7/pipelines.yaml")
print(list(available_pipelines.keys()))


['parse_plaintext', 'parse_sentlines', 'parse_wslines', 'parse_conllu', 'tokenize', 'parse_noisytext']


In [6]:
#this one will take long on first run because of loading the model
p=Pipeline(available_pipelines["parse_plaintext"])
parsed=p.parse("Minulla on ruskea koira! Se haukkuu ja juoksee. Voi että!")

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
Dataset reader: <class 'tnparser.udify.dataset_readers.universal_dependencies.UniversalDependenciesDatasetReader'>
0it [00:00, ?it/s]Your label namespace was 'upos'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
Your label namespace was 'xpos'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
Your label namespace was 'feats'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentati

In [ ]:
print(parsed)

1	Minulla	minä	PRON	_	Case=Ade|Number=Sing|Person=1|PronType=Prs	0	root	_	_
2	on	olla	AUX	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	1	cop:own	_	_
3	ruskea	ruskea	ADJ	_	Case=Nom|Degree=Pos|Number=Sing	4	amod	_	_
4	koira	koira	NOUN	_	Case=Nom|Number=Sing	1	nsubj:cop	_	_
5	!	!	PUNCT	_	_	1	punct	_	_

1	Se	se	PRON	_	Case=Nom|Number=Sing|PronType=Dem	2	nsubj	_	_
2	haukkuu	haukkua	VERB	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	0	root	_	_
3	ja	ja	CCONJ	_	_	4	cc	_	_
4	juoksee	juosta	VERB	_	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	2	conj	_	_
5	.	.	PUNCT	_	_	2	punct	_	_

1	Voi	voi	INTJ	_	_	0	root	_	_
2	että	että	INTJ	_	_	1	fixed	_	_
3	!	!	PUNCT	_	_	1	punct	_	_




# GPU mode

* The pipeline runs by default in CPU mode
* Needs to be told to run in GPU
* This is a bit tricky right now but not impossible
* Note: if you now switch the Runtime into GPU, you need to re-run the pip install


In [7]:
#I do realize this ain't good! :)
import types
extra_args=types.SimpleNamespace()
extra_args.__dict__["udify_mod.device"]="0" #simulates someone giving a --device 0 parameter to Udify
extra_args.__dict__["lemmatizer_mod.device"]="0" 

p=Pipeline(available_pipelines["parse_plaintext"],extra_args)
parsed=p.parse("Minulla on ruskea koira! Se haukkuu ja juoksee. Voi että!")
print("Parsed has this many lines:",len(parsed.split("\n")))


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
Dataset reader: <class 'tnparser.udify.dataset_readers.universal_dependencies.UniversalDependenciesDatasetReader'>
0it [00:00, ?it/s]Your label namespace was 'upos'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
Your label namespace was 'xpos'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
Your label namespace was 'feats'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentati

Parsed has this many lines: 25


In [8]:
#Since we are on a GPU, we can try to push through quite a bit more of data
parsed=p.parse("Minulla on ruskea koira! Se haukkuu ja juoksee. Voi että! "*200) #takes forever on CPU, finishes in few seconds on GPU
print("Parsed has this many lines:",len(parsed.split("\n")))

Parsed has this many lines: 4403


# Process the output

* The output of the pipeline run is a conll-u string
* You can parse it in any number of ways
* This is my preferred:

In [9]:
ID,FORM,LEMMA,UPOS,XPOS,FEAT,HEAD,DEPREL,DEPS,MISC=range(10) #the 10 columns

def read_conll(inp,max_sent=0,drop_tokens=True,drop_nulls=True):
    """
    inp: list of lines or an open file
    max_sent: 0 for all, >0 to limit
    drop_tokens: ignore multiword token lines
    drop_nulls: ignore null nodes in enhanced dependencies

    Yields lines of the parse and comments
    """

    comments=[]
    sent=[]
    yielded=0
    for line in inp:
        line=line.rstrip("\n")
        if line.startswith("#"):
            comments.append(line)
        elif not line:
            if sent:
                yield sent,comments
                yielded+=1
                if max_sent>0 and yielded==max_sent:
                    break
                sent,comments=[],[]
        else:
            cols=line.split("\t")
            if drop_tokens and "-" in cols[ID]:
                continue
            if drop_nulls and "." in cols[ID]:
                continue
            sent.append(cols)
    else:
        if sent:
            yield sent,comments

for one_sent,comments in read_conll(parsed.split("\n"),5):
    words=(word_line[FORM] for word_line in one_sent)
    lemmas=(word_line[LEMMA] for word_line in one_sent)
    print(" ".join(words))
    print(" ".join(lemmas))
    print()

# and that's really all there is to it :)


Minulla on ruskea koira !
minä olla ruskea koira !

Se haukkuu ja juoksee .
se haukkua ja juosta .

Voi että !
voi että !

Minulla on ruskea koira !
minä olla ruskea koira !

Se haukkuu ja juoksee .
se haukkua ja juosta .

